In [1]:
from pathlib import Path

import pandas as pd
from pyproj import CRS

from bedrock.gi.ags.read import ags_to_dfs
from bedrock.gi.ags.schemas import (
    Ags3CORE,
    Ags3GEOL,
    Ags3HOLE,
    Ags3ISPT,
    Ags3WETH,
    BaseSAMP,
)
from bedrock.gi.ags.transform import ags3_to_brgi
from bedrock.gi.concatenate import concatenate_databases
from bedrock.gi.validate import check_no_gis_brgi_database

# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)

In [2]:
cwd = Path.cwd()
gi_dir = cwd / "gi-data" / "kaitak"
gi_files = [
    gi_dir / "31241" / "GE9908.7.ags",
    gi_dir / "44751" / "GE-2005-03-57 rev0.ags",
    gi_dir / "47615" / "GE-2007-13-4 rev0.ags",
]
gi_files

[WindowsPath('c:/Users/joost/ReposWindows/bedrock-gi/sandbox/gi-data/kaitak/31241/GE9908.7.ags'),
 WindowsPath('c:/Users/joost/ReposWindows/bedrock-gi/sandbox/gi-data/kaitak/44751/GE-2005-03-57 rev0.ags'),
 WindowsPath('c:/Users/joost/ReposWindows/bedrock-gi/sandbox/gi-data/kaitak/47615/GE-2007-13-4 rev0.ags')]

In [3]:
crs = CRS(2326)
output_excel_path = cwd / "kaitak_gi.xlsx"

In [4]:
brgi_db = {}
for gi_file in gi_files:
    with open(gi_file) as ags_file:
        ags3_data = ags_file.read()
    ags3_db = ags_to_dfs(ags3_data)
    brgi_db_from_1_ags_file = ags3_to_brgi(ags3_db, crs)
    brgi_db = concatenate_databases(brgi_db, brgi_db_from_1_ags_file)

check_no_gis_brgi_database(brgi_db)

AGS 3 data was read for Project GE/99/08.7
This Ground Investigation data contains groups:
['PROJ', 'HOLE', 'PTIM', 'SAMP', 'GEOL']

Transforming AGS 3 groups to Bedrock tables...

'Project' table was created successfully.

'Location' table was created successfully.

'Sample' table was created successfully.

'InSitu_PTIM' table was created successfully.

'InSitu_GEOL' table was created successfully.

AGS 3 data was read for Project GE/2005/03.57
This Ground Investigation data contains groups:
['PROJ', 'HOLE', 'CDIA', 'CORE', 'DETL', 'FLSH', 'GEOL', 'HDIA', 'PTIM', 'SAMP', '?LEGD', 'UNIT', 'ABBR', 'DICT']

Transforming AGS 3 groups to Bedrock tables...

'Project' table was created successfully.

'Location' table was created successfully.

'Sample' table was created successfully.

'InSitu_CDIA' table was created successfully.

'InSitu_CORE' table was created successfully.

'InSitu_DETL' table was created successfully.

'InSitu_FLSH' table was created successfully.

'InSitu_GEOL' table wa

True

In [13]:
for table in brgi_db.values():
    depth_to_top = table.get("depth_to_top")
    if depth_to_top is not None:
        print(int(depth_to_top.isnull().sum()))

0
0
0
0
0
0
0
0
0
0
0
0


In [9]:
# Validate raw AGS 3 data with pandera schemas.
# Use try-catch to avoid crashing the notebook, while still displaying the errors in the data.

try:
    Ags3HOLE.validate(ags3_db["HOLE"])
except Exception as e:
    print(e)

try:
    BaseSAMP.validate(ags3_db["SAMP"])
except Exception as e:
    print(e)

try:
    Ags3GEOL.validate(ags3_db["GEOL"])
except Exception as e:
    print(e)

try:
    Ags3ISPT.validate(ags3_db["ISPT"])
except Exception as e:
    print(e)

try:
    Ags3CORE.validate(ags3_db["CORE"])
except Exception as e:
    print(e)

try:
    Ags3WETH.validate(ags3_db["WETH"])
except Exception as e:
    print(e)

'HOLE'
'SAMP'
non-nullable series 'GEOL_GEOL' contains null values:
76     None
137    None
241    None
267    None
Name: GEOL_GEOL, dtype: object
Error while coercing 'ISPT_NVAL' to type int64: Could not coerce <class 'pandas.core.series.Series'> data_container into type int64:
     index failure_case
0       14         None
1       26         None
2       59         None
3       86         None
4       87         None
..     ...          ...
144    556         None
145    557         None
146    558         None
147    574         None
148    593         None

[149 rows x 2 columns]
Error while coercing 'CORE_SREC' to type int64: Could not coerce <class 'pandas.core.series.Series'> data_container into type int64:
    index failure_case
0       0         None
1       1         None
2       2         None
3       3         None
4       4         None
..    ...          ...
56    292         None
57    293         None
58    294         None
59    295         None
60    296         None

In [11]:
write_gi_dfs_to_excel(ags3_db, output_excel_path)

Excel sheet names cannot contain [':', '/', '\\', '?', '*', '[', ']']. Replaced '?LEGD' with '_LEGD'
Ground Investigation data has been written to 'c:\Users\joost\ReposWindows\bedrock-gi\sandbox\kaitak_gi.xlsx'.


# Misc

In [ ]:
def prepend_column(df: pd.DataFrame, column_name: str, values) -> pd.DataFrame:
    """Make sure that pd.DataFrame.insert() doesn't cause errors when the column already exists.

    Args:
        df (pd.DataFrame): _description_
        column_name (str): _description_
        values (_type_): _description_

    Returns:
        pd.DataFrame: _description_
    """
    if column_name in df.columns:
        df.drop(columns=column_name, inplace=True)
    df.insert(loc=0, column=column_name, value=values)
    return df

# Speckle Stuff

In [ ]:
import datetime as dt

from dotenv import load_dotenv
from specklepy.api import operations
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.objects import Base
from specklepy.objects.geometry import Line, Point
from specklepy.transports.server import ServerTransport


def to_speckle():
    load_dotenv()
    stream_id = "7fbe8ed384"
    hole_table_path = "data/1_split2/HOLE.csv"
    df = pd.read_csv(hole_table_path, index_col=0)
    client, stream_id = get_stream(stream_id)

    # next create a server transport - this is the vehicle through which you will send and receive
    transport = ServerTransport(client=client, stream_id=stream_id)

    hash = create_hash(df, transport)

    commit_hash(hash, client, stream_id)


def get_stream(stream_id):
    # Authenticate with Speckle server
    speckle_server = "app.speckle.systems"
    speckle_token = os.environ["speckle_token"]
    client = SpeckleClient(host=speckle_server)
    account = get_default_account()

    client.authenticate_with_token(speckle_token)

    # create a new stream. this returns the stream id
    if not stream_id:
        stream_id = client.stream.create(name="a shiny new stream")

    # use that stream id to get the stream from the server
    new_stream = client.stream.get(id=stream_id)
    return client, stream_id


def create_hash(df, transport):
    newObj = Base()
    for i, row in df.iterrows():
        x = row["HOLE_NATE"]
        y = row["HOLE_NATN"]
        z_top = row["HOLE_GL"]
        z_bot = z_top - row["HOLE_FDEP"]

        # GisPointElement, GisLineElement lijken niet te werken
        p1 = Point(x=x, y=y, z=z_top)
        p2 = Point(x=x, y=y, z=z_bot)
        line = Line(start=p1, end=p2)
        relevant_cols = [
            "HOLE_ID",
            "HOLE_TYPE",
            "HOLE_STAR",
            "HOLE_LOG",
            "?HOLE_DLOG",
            "?HOLE_CHEK",
            "?HOLE_DCHK",
            "HOLE_REM",
            "?HOLE_FLSH",
            "HOLE_ENDD",
            "HOLE_BACD",
            "HOLE_CREW",
            "HOLE_INCL",
            "HOLE_EXC",
        ]

        for col in relevant_cols:
            line[col] = row[col]

        newObj[f"myline{i}"] = line
        break

    # this serialises the block and sends it to the transport
    hash = operations.send(base=newObj, transports=[transport])
    return hash


def commit_hash(hash, client, stream_id):
    # you can now create a commit on your stream with this object
    commid_id = client.commit.create(
        stream_id=stream_id,
        object_id=hash,
        message=f"these are lines I made in speckle-py at {dt.datetime.now()}",
    )


if __name__ == "__main__":
    to_speckle()
